<a href="https://colab.research.google.com/github/yarkalm/def_metallization/blob/main/Metallization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import math
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from itertools import product

import sklearn
from tensorflow import keras
from keras.layers import Dense, Dropout
import matplotlib.pyplot as plt
from keras import metrics
import tensorflow as tf

In [ ]:
df = pd.read_excel('/content/Датасет_металлург.xlsx')



input_data = df[["CO2","CO","H2","Herm","Magn","Temp","Time","m before","m after"]]
output_data = df[["RedDegr"]]

input_data

In [13]:
!git commit

fatal: not a git repository (or any of the parent directories): .git
